<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch07/section_62_63.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ロジスティック回帰モデル
ロジスティック回帰モデルとは
パーセプトロンの欠点である、分類器がクラス判定した際の確からしさの解釈を想定していないことに対処できるモデル。
ロジスティック回帰は二値分類問題に用いられる統計モデルであり、判定の確からしさが確率として表される。
$$
P(\hat{y}=1 \mid x)=\frac{1}{1+\exp\left(-(w^\top x+b)\right)}
$$
wは重みベクトル、bはバイアスと呼ばれるパラメータである
$$
\sigma(a) = \frac{1}{1 + \exp(-a)}
$$
$$
P(\hat{y}=1 \mid x) = \sigma(w^\top x + b)
$$
と書かれることもある。
二番目の式は標準ロジスティック関数、標準シグモイド関数とも呼ばれている。
positive、negative判定の二値分類問題の場合はpositiveの確率を標準シグモイド関数によって求めたらもう片方の確率も自然と出てくる。
単純にシグモイド関数の0.5の値を境界に設定するとaの値が正か負かでpositive, negativeが決定する。
その場合は
$$
w^\top x + b \ge 0
$$
となっているので、bを右辺に移行すると
$$
w^\top x \ge -b
$$
となり、しきい値b（バイアス）によって分類が行われていることになる

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import pandas as pd

In [68]:
train_path = "/content/drive/MyDrive/SST-2/train.tsv"
dev_path = "/content/drive/MyDrive/SST-2/dev.tsv"

train_df = pd.read_csv(train_path, sep='\t')
dev_df = pd.read_csv(dev_path, sep='\t')

In [69]:
def row_to_vec(df):
  tokens = df['sentence'].split()
  feature = {}
  for token in tokens:
    if token not in feature:
      feature[token] = 1
    else:
      feature[token] += 1

  return {
      'text': df['sentence'],
      'label': df['label'],
      'feature': feature
  }

各行ごとのベクトル情報を要素にもつSeriesを作成した後、pythonのlist形式に変換する

In [70]:
X_train_data = train_df.apply(row_to_vec, axis=1).tolist()
X_dev_data = dev_df.apply(row_to_vec, axis=1).tolist()

各データから学習と検証に使用するfeature dataとlabel dataを配列形式で取り出す

In [71]:
X_train_dicts = [ex['feature'] for ex in X_train_data]
y_train = [ex['label'] for ex in X_train_data]

X_dev_dicts = [ex['feature'] for ex in X_dev_data]
y_dev = [ex['label'] for ex in X_dev_data]

In [72]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## DictVectorizerとは
pythonのdictオブジェクトのリストとして表される特徴量配列をscikit-learn推定器で使用されるNumPy / SciPy表現に変換するために使用できる。
値に加えて特徴名を格納できるという利点がある。

各事例は「特徴名 → 数値」（例：単語 → 出現回数）の辞書として表現され、
DictVectorizer は学習データ全体から特徴名（語彙）を収集し、
それぞれの特徴に対応する次元（行列の列）を割り当てることで特徴行列を構築する。

このとき、特徴名と行列の列との対応関係が内部に保持されるため、
学習後に各特徴に対応する重みを確認し、モデルの判断を解釈できるという利点がある。

特に Bag of Words のように高次元かつ多くの要素が 0 となる特徴表現においては、
`sparse=True` を指定することで疎行列として保持され、
0 の要素を明示的に保存せずに計算できるため、
メモリ効率および計算効率の向上が可能である。

Bag of Words（BoW）とは、文章中の単語の出現回数（または出現有無）を特徴量として用いる手法である。
単語の語順や文脈は考慮せず、文を「単語の集合（袋）」として扱い、各単語の頻度でベクトル表現する。



In [73]:
vec = DictVectorizer(sparse=True) # DictVectorizer class instanceを作成する

fit_transform と transform の違い

fit_transform は train data を用いて、特徴名（語彙）を収集し、
特徴名→列番号（次元）の対応ルールを作成（fit）した上で、
そのルールに基づいて各事例を特徴ベクトル（行列）に変換（transform）する。

transform は、すでに fit で作成した対応ルールを使って、
各事例を特徴ベクトル（行列）に変換するだけである。
このとき、学習時に存在しなかった特徴（未知語）は無視され、0として扱われる。


In [74]:
X_train = vec.fit_transform(X_train_dicts)
X_dev = vec.transform(X_dev_dicts)

## LogisticRegressionとは
ロジスティック回帰を学習しているモデル。
max_iterとは最適化の繰り返し回数の上限である。
ロジスティック回帰は損失を小さくするために少しずつw, bを更新していく。
最大max_iter回更新をして、収束したら途中で止まる。
solver='liblinear'とはどのアルゴリズム（最適化手法）でw, bを求めるかの指定である。

In [75]:
clf = LogisticRegression(max_iter=1000, solver='liblinear')

fitは何をしているのか
1. 確率を計算する
2. 損失を計算する（損失は基本的に0以上となる）
$$
L(w,b)
= -\sum_{i=1}^{n}\Bigl[
y_i \log p_i + (1-y_i)\log(1-p_i)
\Bigr]
$$
3. 損失が小さくなるようにw, bを更新する



In [76]:
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [77]:
pred = clf.predict(X_dev)
print("dev accuracy:", accuracy_score(y_dev, pred))

dev accuracy: 0.8107798165137615
